In [1]:
import AmazonReviews

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
ar = AmazonReviews.AmazonReviews()

PATH = '../data/amazon_reviews_us_Toys_v1_00.tsv'
ar.load_data(PATH)
ar.calc_trend_score()

ar.create_observations()

In [ ]:
ar.create_train_test_split()

## DTM Creation

Create DTM and only re

In [53]:
from nltk.corpus import words

words_corpus = set(words.words())
analyzer = CountVectorizer().build_analyzer()

def english_corpus(doc):
    return [w for w in analyzer(doc) if w in words_corpus]

ar.add_dtm(CountVectorizer(stop_words='english', analyzer=english_corpus),'count_1_gram')

In [54]:
features = ar.X_train['count_1_gram'][1]

In [55]:
len(features)

24462

In [56]:
features

['aa',
 'aam',
 'aback',
 'abacus',
 'abalone',
 'abandon',
 'abandoned',
 'abas',
 'abate',
 'abb',
 'abbess',
 'abbey',
 'abbot',
 'abbreviate',
 'abbreviation',
 'abdomen',
 'abdominal',
 'abduction',
 'abed',
 'aberrant',
 'aberration',
 'abhor',
 'abhorrent',
 'abide',
 'abigail',
 'ability',
 'ablaze',
 'able',
 'ably',
 'abnegation',
 'abnormal',
 'abnormality',
 'abnormally',
 'aboard',
 'abody',
 'abominable',
 'abomination',
 'abort',
 'aborted',
 'abortion',
 'abound',
 'about',
 'abouts',
 'above',
 'abrade',
 'abrasion',
 'abrasive',
 'abreast',
 'abroad',
 'abrupt',
 'abruptly',
 'absconded',
 'absence',
 'absent',
 'absinthe',
 'absolute',
 'absolutely',
 'absorb',
 'absorbable',
 'absorbed',
 'absorbency',
 'absorbent',
 'absorber',
 'absorbing',
 'absorption',
 'absorptive',
 'abstract',
 'abstraction',
 'abstractness',
 'absurd',
 'absurdity',
 'absurdly',
 'abu',
 'abundance',
 'abundant',
 'abundantly',
 'abuse',
 'abuser',
 'abusive',
 'abut',
 'abysmal',
 'abyss',

In [27]:
features.sort(reverse=True)

In [32]:
ar.reviews_selected_df.marketplace.unique()

array(['US'], dtype=object)

In [35]:
len(words.words())

236736

In [36]:
len(features)

93756

In [38]:
words_corpus = set(words.words())
feature_english = [f for f in features if f in words_corpus]

In [39]:
len(feature_english)

24165

In [40]:
ar.obs.shape

(1781440, 5)

In [41]:
len(words_corpus)

235892

In [43]:
feature_english

['zoom',
 'zoo',
 'zoned',
 'zone',
 'zonal',
 'zombie',
 'zoll',
 'zoid',
 'zoic',
 'zodiac',
 'zo',
 'zirconium',
 'zirconia',
 'zippy',
 'zipping',
 'zipper',
 'zip',
 'zing',
 'zinfandel',
 'zinc',
 'zigzagged',
 'zigzag',
 'zig',
 'zeta',
 'zesty',
 'zest',
 'zero',
 'zeppelin',
 'zephyr',
 'zenana',
 'zee',
 'zed',
 'zebra',
 'zealous',
 'zeal',
 'zany',
 'zander',
 'zain',
 'zagged',
 'zag',
 'zac',
 'za',
 'yus',
 'yurt',
 'yummy',
 'yuletide',
 'yule',
 'yuh',
 'yucky',
 'yuck',
 'yuan',
 'yr',
 'yoy',
 'yow',
 'youve',
 'youthful',
 'youth',
 'youse',
 'youngster',
 'youngish',
 'younger',
 'young',
 'youd',
 'yorker',
 'york',
 'yore',
 'yor',
 'yonder',
 'yond',
 'yon',
 'yolked',
 'yolk',
 'yoke',
 'yok',
 'yoi',
 'yogi',
 'yoga',
 'yodel',
 'yo',
 'ym',
 'yin',
 'yielding',
 'yield',
 'yesteryear',
 'yesterday',
 'yester',
 'yest',
 'yes',
 'yer',
 'yep',
 'yeoman',
 'yen',
 'yelp',
 'yellowy',
 'yellows',
 'yellowly',
 'yellowish',
 'yellowing',
 'yellow',
 'yelling',
 '